## Ejemplo de uso de Spark y Jupyter

El objetivo del siguiente Notebook es mostrar la integración de Jupyter con Spark en Ecloud. 


* Obtener el Contexto de Spark: 

Para cada ejecución en spark el notebook debe enlazarse a un contexto que retorna el Spark. A continuación podemos visualizar el contexto de Spark

In [ ]:
sc

* Paralelizar un conjunto de datos: 
    
    Con la función parallelize, el Spark hace una división del conjunto de datos entre los esclavos que integran el clúster de Spark. Luego hace la función collect que devuelve los resultados de los workers. Para el ejemplo se envian10 elementos y se obtienen 10 elementos de los workers.
    
    Para mayor información se puede visitar [Spark Guide](http://spark.apache.org/docs/2.1.1/programming-guide.html)

In [ ]:
a = sc.parallelize(range(10)).collect()
a

* Programar la función PI: 
Este ejemplo permite demostrar el paralelismo de Spark utilizando los workers para computar el cálculo de PI. En este caso se paraleliza el número de particiones entre 10. Esto permite crear 10 RDD (Resiliance Distributed Dataset), y luego hace el mapeo de los nodos y finalmente hace una operación de reduce para obtener el dato final de PI.

Para mayor información sobre RDD visitar [RDD Spark](https://www.tutorialspoint.com/apache_spark/apache_spark_rdd.htm)

In [ ]:
import sys
from random import random
from operator import add
import time

partitions = 10
n = 100000 * partitions

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 < 1 else 0

inicio = time.time()
count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))

fin = time.time()

print("Tiempo: " + str((fin - inicio)) + " Sec")

*  Ejemplo de PageRank: 

    En este  ejemplo de Spark  se comienza leyendo  un fichero con un conjunto de URLs e inicializando los vecinos. 
    Actualiza continuamente las URls consiguiendo una distribución de probabilidad que representa la probabilidad de que una persona haga clic aleatoriamente en vínculos web  y luego lleguen a una página web en particular. Si ejecutamos el programa PageRank con el archivo de datos de entrada e indicamos 10 iteraciones, obtendremos la salida que se presenta al ejecutar el código. 
    
    

In [ ]:
from __future__ import print_function

import re
import sys
from operator import add

from pyspark.sql import SparkSession


def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]


lines = spark.read.text("file:///eslap/legacy/spark/data/mllib/pagerank_data.txt").rdd.map(lambda r: r[0])

# Loads all URLs from input file and initialize their neighbors.

links = lines.map(lambda urls: parseNeighbors(urls)).distinct().groupByKey().cache()

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.

ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(10):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)

    # Collects all URL ranks and dump them to console.
    for (link, rank) in ranks.collect():
        print("%s has rank: %s." % (link, rank))
